Links útiles

+ [Installing_VisIt.html](https://visit-sphinx-github-user-manual.readthedocs.io/en/develop/getting_started/Installing_VisIt.html)
+ [manual_v3.3.1.pdf](https://visit-sphinx-github-user-manual.readthedocs.io/_/downloads/en/v3.3.1/pdf/)

Pra instalar Visit realizamos el siguiente procedimiento

Ingresamos a la siguiente página

+ [releases-as-tables.html](https://visit-dav.github.io/visit-website/releases-as-tables/)

Aquí descargamos la version de Visit 3.3.1 para ubuntu 20 (ya que esta puede correr en Ubuntu 22) y también descargamos el script de instalación `visit-install`. Estos dos archivos los ubicamos en una misma carpeta y damos permisos de ejecución al script de instalación de la siguiente manera,

```bash
    chmod +x visit-install3_3_1
```

Luego ejecutamos el script, seleccionando la versión a instalar y la arquitectura (este nombre debe coincidir con el nombre del directorio comprimido descargado *.tgz)

```bash
    ./visit-install3_3_1 3.3.1 linux-x86_64-ubuntu20 /usr/local/visit
```

Luego creamos un alias para ejecutar visit en background

```bash
    $ vi ~/.bash_aliases

    #agregamos la siguiente linea para que el comando corra fuera de bash shell
    alias visit='/usr/local/visit/bin/visit 2>/dev/null &'
```

# Ejemplo de resolución del problema de Poisson con distintas condiciones de contorno y manipulación de la solución para el problema de cálculo de capacitancias #


El problema a resolver es:


\begin{align}
-\Delta u &= f \;\;\;\;\; \text{in } \Omega \\
u &= g \;\;\;\;\; \text{in } \partial \Omega_{int} \\
\hat{n} \cdot \nabla u &= h \;\;\;\;\; \text{in } \partial \Omega_{ext} \\
\end{align}

Para ello impondremos la versión débil del mismo:

Encuentre $u$ en $H^1(\Omega, f))$ (o sea con las condiciones de contorno de Dirichlet en $\partial \Omega_{int}$) tal que,

$$
\int_{\Omega} \nabla v \cdot \nabla u \; d\Omega 
- \int_{\Omega} v \; f \; d\Omega 
- \oint_{\partial \Omega_{ext}} v \; h \; d\Gamma 
= 0 \;\;\;\;\; \forall v \;\; \in H^1_0(\Omega)
$$

Si obtenemos un $u$ satisfaciendo esta ecuación, y es suficientemente suave, entonces podemos integrar por partes el primer término y obtener:

$$
\int_{\Omega}  v \; (-\Delta u - f) \; d\Omega 
+ \oint_{\partial \Omega_{ext}} v \; (\hat{n} \cdot \nabla u - h) \; d\Gamma 
= 0 \;\;\;\;\; \forall v \;\; \in H^1_0(\Omega)
$$

Tomando $v$ arbitrario pero de soporte compacto vemos que $u$ debe satisfacer:

$$
-\Delta u = f \;\;\;\;\; \text{in } \Omega,
$$
y tomando $v$ arbitrario vemos que también se debe cumplir la condición de Neumann,

$$
\hat{n} \cdot \nabla u = h \;\;\;\;\; \text{in } \partial \Omega_{ext}.
$$

La condición de Dirichlet es automática por la elección del espacio.

Luego utilizaremos la solución encontrada para un problema de capacitancia.




Para resolver el problema utilizaremos la infraestructura del paquete `Gridap.jl` de Julia. Este ejemplo es una recopilación de varios ejemplos en el tutorial del paquete. 

In [77]:
import Pkg; Pkg.activate("gridap_makie");

create_directories = false;
if create_directories
    mkdir("models");
    mkdir("images");
end

  Activating project at `~/github_repositories/my_repositories/pde2022/tareas/05_tarea/gridap_makie`


In [78]:
# Pkg.instantiate();

In [79]:
using Gridap;
using FileIO;       #Gráficos y salidas
using GridapGmsh;
#using gmsh #]add https://github.com/koehlerson/gmsh.jl.git
using Gmsh;

In [80]:
using Pkg
Pkg.status("Gridap")
Pkg.update("Gridap")

No Matches in `~/github_repositories/my_repositories/pde2022/tareas/05_tarea/gridap_makie/Project.toml`


    Updating registry at `~/.julia/registries/General.toml`


  No Changes to `~/github_repositories/my_repositories/pde2022/tareas/05_tarea/gridap_makie/Project.toml`
  No Changes to `~/github_repositories/my_repositories/pde2022/tareas/05_tarea/gridap_makie/Manifest.toml`


In [81]:
#plot_s = false #true
plot_s = true
if plot_s
    # using Pkg; Pkg.activate("./gridap_makie")
    # Pkg.add(Pkg.PackageSpec(;name="Makie", version="0.18.2"));
    # import Pkg; Pkg.add("GridapMakie");
    using Makie;
    #import Pkg; Pkg.precompile()
    using GridapMakie,GLMakie;  #Para graficar 
end

Vamos a usar grillas construidas con la librería `gmsh`, a través del script ```mesh_generator.jl```. Notar que en el script se da nombre a las tres fronteras, la externa (rectangular), `"ext"` y las internas `"inner_circle"` y `"inner_square"`. En base al script, y siguiendo el tutorial de `gmsh`, podrán construir otras grillas. También se pueden usar otras librerías para construir grillas. Estas se importan a la infraestructura **Gridap** y con ellas se construye la triangulación a usar. 

In [82]:
include("mesh_generator.jl")

make_model (generic function with 1 method)

In [83]:
grid_type = "rectangle_hole_square"

#Resoluciones:
res = 1

if res == 1
    lc = 1e-1
    lc_f = 0.25e-1
    name = grid_type * "_coarse"
elseif res == 2 
    lc = 5e-2
    lc_f = 2.5e-2
    name = grid_type * "_intermediate"
elseif res == 3
    lc = 1e-2
    lc_f = 0.25e-2
    name = grid_type * "_finner"
end
    
#Lados exteriores de la grilla rectangular
side_x = 2
side_y = 1

#Rectangulo interior
rec_base = 0.25  #Coordenada y de la base
rec_top = 0.75   #Coordenada y del lado superior
rec_left = 1.25  #Coordenada x del lado izquierdo
rec_right = 1.75 #Coordenada x del lado derecho

#Circulo interior
circ_center_x = 0.5  #Coordenada x del centro
circ_center_y = 0.5  #Coordenada y del centro
circ_radius = 0.25   #Radio



p = (name, side_x, side_y, circ_center_x, circ_center_y, circ_radius, rec_base, rec_top, rec_left, rec_right, lc, lc_f)

("rectangle_hole_square_coarse", 2, 1, 0.5, 0.5, 0.25, 0.25, 0.75, 1.25, 1.75, 0.1, 0.025)

In [84]:
model = make_model(grid_type, p)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Circle)
Info    : [ 50%] Meshing curve 6 (Circle)
Info    : [ 50%] Meshing curve 7 (Circle)
Info    : [ 60%] Meshing curve 8 (Circle)
Info    : [ 70%] Meshing curve 10 (Line)
Info    : [ 80%] Meshing curve 11 (Line)
Info    : [ 90%] Meshing curve 12 (Line)
Info    : [100%] Meshing curve 13 (Line)
Info    : Done meshing 1D (Wall 0.0010979s, CPU 0.001068s)
Info    : Meshing 2D...
Info    : Meshing surface 1000 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0318202s, CPU 0.031817s)
Info    : 964 nodes 1941 elements
Info    : Writing 'models/rectangle_hole_square_coarse.msh'...
Info    : Done writing 'models/rectangle_hole_square_coarse.msh'
Info    : Reading 'models/rectangle_hole_square_coarse.msh'...
Info    : 26 entities
Info    : 963 nodes
Info    : 1928 ele

UnstructuredDiscreteModel()

In [85]:
Ω = Triangulation(model)

BodyFittedTriangulation()

In [86]:
degree = 3
dΩ = Measure(Ω,degree)

Measure()

In [ ]:
if plot_s
    fig, ax = plot(Ω)
    ax.aspect = AxisAspect(2)
    wireframe!(Ω, color=:black, linewidth=1)
    scatter!(Ω, marker=:star8, markersize=4, color=:blue)
    fig
end

Tambén podemos graficar los contornos:

In [ ]:
boundary_tags = ["inner_circle", "inner_square", "ext"]

Γ = BoundaryTriangulation(model,tags=boundary_tags)
dΓ = Measure(Γ,degree)

if plot_s
    fig, ax = plot(Γ, linewidth=8)
    ax.aspect = AxisAspect(2)
    wireframe!(Γ, color=:black, linewidth=1)
    fig
end

Vamos a elegir dos problemas simples para calcular la matríz de capacitancia de un conjunto de conductores. Tomaremos a los conductores como los dos cuerpos: el círculo y el cuadrado y pondremos condiciones de potencial constante. El borde externo se tomará como *infinito* y pondremos allí siempre potencial cero.

In [89]:
capacity_cs = false # potencial 1 en el círculo y potencial 0 en el cuadrado.
capacity_sc = false # potencial 0 en el círculo y potencial 1 en el cuadrado.
capacity_cs = true
#capacity_sc = true 

true

Una vez que tenemos el grillado comenzamos a definir los elementos finitos que utilizaremos. En este caso usaremos elementos lagrangiano de **orden 1** que cumplirán una condición de Dirichlet en la región $\partial \Omega_{int}$. Al construirse la grilla esta región ha sido marcada como la frontera interior del rectángulo con el `tag` `"int"`. 

In [90]:
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)

dirichlet_tags= ["inner_circle", "inner_square","ext"] 

V = TestFESpace(model,reffe;conformity=:H1,dirichlet_tags = dirichlet_tags)
#V = TestFESpace(model,reffe;conformity=:L2,dirichlet_tags = dirichlet_tags) #no funciona la inversión. 

UnconstrainedFESpace()

Asignamos los valores de contorno para los potenciales. 

In [91]:
# internal Dirichlet boundary condition
g(x) = 1.0 # esta puede ser una función de x (vector posición)
if capacity_cs
    U = TrialFESpace(V,[g 0.0 0.0])
elseif capacity_sc
    U = TrialFESpace(V,[0.0 g 0.0])
end

TrialFESpace()

A continuación definimos el problema débil en forma abstracta:

In [92]:
f(x) = 0 # en este caso la fuente es cero, pero se puede poner una distribución de carga.

a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ  # en a(u,v) va toda la dependencia con u que es la incógnita. 

b(v) = ∫(v*f )*dΩ # aquí todo lo que es fuente. 


b (generic function with 1 method)

A partir de este punto el paquete **Gridap.jl** genera un sistema del tipo $Ax=b$ y lo resuelve para la versión elementos finitos de u.

In [93]:
op = AffineFEOperator(a,b,U,V)

AffineFEOperator()

In [94]:
ls = LUSolver()
lb = BackslashSolver() # x = A \ b
solver = LinearFESolver(lb)

LinearFESolver()

In [95]:
uh = solve(solver,op)
#uh = solve(op)

SingleFieldFEFunction():
 num_cells: 1724
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 10701547684115738716

Ahora podemos graficar la solución encontrada y guardarla además en un archivo que se puede leer con paraview o visit.

In [ ]:
if plot_s 
    fig, ax, plt = plot(Ω, uh)
    ax.aspect = AxisAspect(2)
    Colorbar(fig[2,1], plt, vertical=false)
    fig
end

In [97]:
if capacity_cs
    writevtk(Ω,"images/solucion_cs_$res",cellfields=["uh_cs_$res" => uh])
    writevtk(Ω,"images/grad_cs_$res",cellfields=["grad_uh_cs_$res" => ∇(uh)])
elseif capacity_sc
    writevtk(Ω,"images/solucion_sc_$res",cellfields=["uh_sc_$res" => uh])
    writevtk(Ω,"images/grad_cs_$res",cellfields=["grad_uh_cs_$res" => ∇(uh)])
end

(["images/grad_cs_1.vtu"],)

Una vez obtenida la solución se pueden calcular algunas cantidades físicamente interesantes, por ejemplo la carga contenida en una región. Aquí calculamos la carga en la región exterior y en la interior. Note el cambio de signo para la interior ya que la normal siempre es toma saliente.

Recordemos que la carga se define como: 
\begin{equation}
Q_i = \int_{\partial \Omega_i} \sigma dS =  \frac{1}{4\pi}\int_{\partial \Omega_i} E \cdot \; dS
\end{equation}

Mientras que la  **matriz de capacidades** como: 

\begin{equation}
Q_i = C_{ij}V^j
\end{equation}

In [98]:
Γ_ext = BoundaryTriangulation(model,tags="ext")
dΓ_ext = Measure(Γ_ext,degree)
nb_ext = get_normal_vector(Γ_ext)
Q_ext = -sum(∫((nb_ext ⋅ ∇(uh)))*dΓ_ext)/4/π

0.5517261137123992

In [99]:
Γ_square = BoundaryTriangulation(model,tags="inner_square")
dΓ_square = Measure(Γ_square,degree)
nb_square = get_normal_vector(Γ_square)
Q_square = sum(∫((nb_square ⋅ ∇(uh)))*dΓ_square)/4/π

-0.05404171311150682

In [100]:
Γ_circle = BoundaryTriangulation(model,tags="inner_circle")
dΓ_circle = Measure(Γ_circle,degree)
nb_circle = get_normal_vector(Γ_circle)
Q_circle = sum(∫((nb_circle ⋅ ∇(uh)))*dΓ_circle)/4/π

0.5689633878818985

In [101]:
Q_ext - Q_circle - Q_square # esto debería dar cero si no hay fuentes (cargas).

0.03680443894200755

![Una imagen con visIt](images/grad_cs_1_mesh0000.jpeg)
![Una imagen con visIt](images/grad_cs_1_pseudocolor0000.jpeg)
![Una imagen con visIt](images/solucion_cs_1_mesh0000.jpeg)
![Una imagen con visIt](images/solucion_cs_1_pseudocolor0000.jpeg)